# How Far Deep Learning Systems for Text Detection and Recognition in Natural Scenes are Affected by Occlusion?

### This is a step-by-step guide to generate the ISTD-OC dataset. 



In [ ]:
#import drive
from google.colab import drive
drive.mount('./gdrive')
%cd "./gdrive/My Drive/Colab Notebooks"

In [ ]:
#list directory
import os
os.chdir(path)

!ls -l

In [ ]:
#generate ISTD-OC crop words for the text detection task 

import cv2
import random
import numpy as np
from random import randint, gauss, randrange



def splitCoordenadas(coordenadas):
    coordenadas = coordenadas.split(',')
    #print("coordenadas",coordenadas)
    return coordenadas[:8]

def gerarCoordenadas(path):
    coordenadas = open(path, 'r', encoding='utf-8-sig')
    coordenadas = coordenadas.read()
    coordenadas = coordenadas.split('\n')
    result = []
    regioes = len(coordenadas)
    print("regioes",regioes)

    
    for i in range (len(coordenadas)-1):
        aux = splitCoordenadas(coordenadas[i])
        print(aux)
        p1 = (int(aux[0]), int(aux[1]))
        p2 = (int(aux[2]), int(aux[3]))
        p3 = (int(aux[4]), int(aux[5]))
        p4 = (int(aux[6]), int(aux[7]))
        result.append(p1)
        result.append(p2)
        result.append(p3)
        result.append(p4)
    #print("result:",result)    
    return result, regioes-1, coordenadas[-1]



def fazLista(coordenadas):
    auxList = []
    lista = []
    aux = 0
    for i in range (0, len(coordenadas)):
        if aux == 4:
            lista.append(auxList)
            auxList = []
            aux = 0
        auxList.append(coordenadas[i])
        aux += 1
    lista.append(auxList)
    #print("lista",lista)
    return lista

def fazDicionario(regioes, lista):
    dic = {}
    for i in range(0, regioes):
        dic[str(i)] = lista[i]

    return dic

def regiaoTexto(pathImg, regiaoEscolhida):
    img = cv2.imread(pathImg)
    box = img[regiaoEscolhida[0][1]:regiaoEscolhida[3][1], regiaoEscolhida[0][0]:regiaoEscolhida[1][0]]
    return box


def fazOclusao(pathImg, box,regiaoEscolhida,img):
    
    #RATIO BASED ON IMAGE SIZE TO TEXT BOX SIZE
    alfa=0.55
    rand=random.gauss(1.0,0.05)
    proporcao_altura_imagem=(box.shape[0]/img.shape[0])
    proporcao_largura_imagem=(box.shape[1]/img.shape[1])

    #OCCLUSION SIZE EQUALS BOX SIZE MULTIPLIED BY PROPORTION
    largura_oclusao= int(alfa*box.shape[1]*rand+(1-alfa)*box.shape[1]*proporcao_largura_imagem)
    altura_oclusao= int(alfa*box.shape[0]*rand+(1-alfa)*box.shape[0]*proporcao_altura_imagem)
    #OFFSET
    offset_altura=box.shape[0]-altura_oclusao
    offset_largura=box.shape[1]-largura_oclusao

    #set occlusion starting point
    max_x_inicial = offset_altura
    max_y_inicial = offset_largura
    ponto_inicial = [random.randint(0,max_x_inicial), random.randint(0,max_y_inicial)]
    
    #add noise to the cut of the image that will be occluded
    gauss = np.random.normal(0,1,img.size)
    gauss = gauss.reshape(img.shape[0],img.shape[1],img.shape[2]).astype('uint8') 
    noise_img = cv2.add(img,gauss)
    #cv2_imshow(noise_img)
   
    #set random point original image
    x2_final = noise_img.shape[0]
    y2_final= noise_img.shape[1]
    ponto_aleat_img=[random.randint(0,x2_final), random.randint(0,y2_final)]
    
    #print("proporcao_altura_imagem: ", proporcao_altura_imagem, "proporcao_largura_imagem: ", proporcao_largura_imagem,
     #   "largura_oclusao: ", largura_oclusao, "altura_oclusao: ", altura_oclusao, 
      #  "max_x_inicial: ", max_x_inicial, "max_y_inicial: ", max_y_inicial,
        #"max_x2_final: ", max_x2_final, "max_y2_final: ", max_y2_final,
       # "ponto_inicial: ", ponto_inicial, #"ponto_final: ", ponto_final, 
        #"offset_altura: ", offset_altura,"offset_largura: ",offset_largura, 
        #"regiaoEscolhida:", regiaoEscolhida, "img.shape: ", img.shape,
        #"box.shape: ", box.shape)

    #occlusion region 
    box[ponto_inicial[0]:ponto_inicial[0]+altura_oclusao,ponto_inicial[1]:ponto_inicial[1]+largura_oclusao,:]= noise_img[ponto_aleat_img[0]:ponto_aleat_img[0]+altura_oclusao, ponto_aleat_img[1]:ponto_aleat_img[1]+largura_oclusao]
    #print(img[ponto_inicial[0]:ponto_final[0] , ponto_inicial[1]:ponto_final[1]])
    #print(img[ponto_aleat_img[0],ponto_aleat_img[1]]) 
    return box
    

for d in range(1,501):
  pathTxt = "OctShuffle-MLT-master/ICDAR/icdar_teste_GT/images/gt_img_"+str(d)+".txt"
  c, regioes,transcricao = gerarCoordenadas(pathTxt)
  l = fazLista(c)
  dic = fazDicionario(regioes, l)
  pathImg= "PSENet/noise_base/icdar_teste/img_"+str(d)+".jpg"
  img=cv2.imread(pathImg)
  
  for i in range(0, regioes):
      
          regiaoEscolhida = dic[str(i)]
          regiao = regiaoTexto(pathImg, regiaoEscolhida)
          if transcricao is not "###" and regiaoEscolhida[2][0] <= img.shape[0] and regiaoEscolhida[2][1] <= img.shape[1]:   
            regiao = fazOclusao(pathImg, regiao,regiaoEscolhida,img)
            img[regiaoEscolhida[0][1]:regiaoEscolhida[3][1], regiaoEscolhida[0][0]:regiaoEscolhida[1][0]] = regiao
                      #img[regiaoEscolhida[0][1]:regiaoEscolhida[3][1], regiaoEscolhida[0][0]:regiaoEscolhida[1][0]] = [255,0,0]
  pathsave="PSENet/noise_base/detection/100/img_"+str(d)+".jpg"
  cv2.imwrite(pathsave,img)